In [ ]:
from __future__ import print_function, division
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
# log in and go to page to get data
driver.get('https://fantasydata.com/user/login')
time.sleep(1);
logIn = driver.find_element_by_name('Email')
logIn.send_keys('rishis1096@gmail.com')
logIn = driver.find_element_by_name('Password')
logIn.send_keys('password') # false password due to code being put online
logIn.send_keys(Keys.RETURN)

In [ ]:
driver.get('https://fantasydata.com/nfl-stats/fantasy-football-leaders?position=2&team=1&season=2017&seasontype=1&scope=2&subscope=1&startweek=1&endweek=17')
time.sleep(1);

In [ ]:
# use beautifulsoup to scrape - starting with player name
innerHTML = driver.execute_script("return document.body.innerHTML")
innerHTML = BeautifulSoup(innerHTML, 'lxml')
innerHTML

In [ ]:
# get the names of the QBs in order
names = [str(link) for link in innerHTML.find_all('a') if 'nfl-stats/player-details' in str(link)]
names = [link.split('>')[1].split('<')[0] for link in names]
names

In [ ]:
# get week numbers in order
weeks = [str(week) for week in innerHTML.find_all('span') if 'dataItem.Week' in str(week)]
weeks = [week.split('>')[1].split('<')[0] for week in weeks]
weeks

In [ ]:
# get opponents in order
opponents = [str(opponent) for opponent in innerHTML.find_all('span') if 'dataItem.Opponent' in str(opponent)]
opponents = [opponent.split('>')[1].split('<')[0] for opponent in opponents]
opponents

In [ ]:
# get data in order
data = [str(data) for data in innerHTML.find_all('td') if 'class=""' in str(data) and 'span' not in str(data)]
data = [data for data in data if 'href' not in data]
data = [data.split('>')[1].split('<')[0] for data in data]
newData = []
for x in range(len(data)//12):
    newData.append(data[12*x:12*x + 11])
for statList in newData:
    if len(statList) == 11:
        statList.append(0)

completions = []
attempts = []
percentage = []
passingYards = []
avgYdsThrow = []
passTDs = []
interceptions = []
passerRating = []
rushAtt = []
rushYds = []
rushAvg = []
rushTDs = []

for x in range(len(newData)):
    completions.append(newData[x][0])
    attempts.append(newData[x][1])
    percentage.append(newData[x][2])
    passingYards.append(newData[x][3])
    avgYdsThrow.append(newData[x][4])
    passTDs.append(newData[x][5])
    interceptions.append(newData[x][6])
    passerRating.append(newData[x][7])
    rushAtt.append(newData[x][8])
    rushYds.append(newData[x][9])
    rushAvg.append(newData[x][10])
    rushTDs.append(newData[x][11])

In [ ]:
# get fantasy points in order
fantasyPoints = [str(fpts) for fpts in innerHTML.find_all('span') if 'dataItem.FantasyPoints' in str(fpts) and 'PerGame' not in str(fpts)]
fantasyPoints = [fpts.split('>')[1].split('<')[0] for fpts in fantasyPoints]
fantasyPoints

In [ ]:
ATL = pd.DataFrame({'Name':names, 'Week':weeks, 'OPP':opponents, 'CMP':completions, 'ATT':attempts, 'PCT':percentage, 'PassYd':passingYards, 'AvgYds/Pass':avgYdsThrow, 'PassTD':passTDs, 'INT':interceptions, 'Rating':passerRating, 'RushAtt':rushAtt, 'RushYds':rushYds, 'AvgYds/Rush':rushAvg, 'RushTD':rushTDs, 'Fantasy Points':fantasyPoints})
ATL